In [22]:
import sys
from pathlib import Path
import pandas as pd
import logging

logging.basicConfig(
    level=logging.INFO, 
    format="%(asctime)s - %(levelname)s - %(message)s"
)

import importlib
import data.cuped

importlib.reload(data.cuped)

# Add parent directory to sys.path
sys.path.append(str(Path().resolve()))

from data.extract import initialize_bigquery_client, extract_data
from data.bigquery_queries import get_marketing_data, get_dps_data
from data.transform import apply_cleanup
from data.cuped import apply_cuped_adjustment
from data.store import store_data_cloud
from utils.dates import get_iso_week_mondays
from utils.summary_stats import summarize_columns

import utils

importlib.reload(utils.summary_stats)


from utils.summary_stats import summarize_columns


In [15]:
entities = (
        'FP_PK','PY_DO'
    )

weeks = ['2025-02-08']
    
project = "logistics-customer-staging"

# # Initialize client
# client = initialize_bigquery_client(project)

# # Generate SQL
mkt_query = get_marketing_data(entities, weeks[0], restaurant_flag='IN')
dps_query = get_dps_data(entities, weeks[0], restaurant_flag='IN')

# # Extract data
df_raw = extract_data(client, mkt_query, dps_query)
print(df_raw.head())

df_raw["as_of_date"] = weeks[0]  

# # Transform data
df_cleaned = apply_cleanup(df_raw)
print(df_cleaned.head())

# # Apply CUPED
# df_cuped = apply_cuped_adjustment(
#     df_cleaned, 
#     pre_post_metric_pairs=[("orders_pre", "orders_post"), 
#                            ("analytical_profit_pre", "analytical_profit_post")]
# )
# print(df_cuped.head())

# # Store (locally only for testing)
# store_data_cloud(df_cuped, weeks, save_local=True, save_cloud_storage=False)


2025-05-14 21:13:13,389 - INFO - Starting data extraction...
2025-05-14 21:15:06,778 - INFO - Data extraction completed in 113.39 seconds.


  brand_name entity_id customer_id  is_customer_holdout  orders_pre  \
0    Pandora     FP_PK    pk5mb3nh                False          47   
1    Pandora     FP_PK    pkw6bqf0                False          53   
2    Pandora     FP_PK    pkabta16                False          21   
3    Pandora     FP_PK    pk98d24q                False          33   
4    Pandora     FP_PK    t9tvl5vd                False          27   

   orders_post  analytical_profit_pre  analytical_profit_post  
0           27              28.202469               17.987374  
1           28              14.348259               12.112171  
2            6              17.580291                6.683795  
3           17               7.918338                6.170878  
4           14              16.837372                6.873295  


2025-05-14 21:15:12,485 - INFO - apply_cleanup(): 0 rows removed. Final dataset size: 31554036


  brand_name entity_id customer_id  is_customer_holdout  orders_pre  \
0    Pandora     FP_PK    pk5mb3nh                False          47   
1    Pandora     FP_PK    pkw6bqf0                False          53   
2    Pandora     FP_PK    pkabta16                False          21   
3    Pandora     FP_PK    pk98d24q                False          33   
4    Pandora     FP_PK    t9tvl5vd                False          27   

   orders_post  analytical_profit_pre  analytical_profit_post  as_of_date  
0           27              28.202469               17.987374  2025-02-08  
1           28              14.348259               12.112171  2025-02-08  
2            6              17.580291                6.683795  2025-02-08  
3           17               7.918338                6.170878  2025-02-08  
4           14              16.837372                6.873295  2025-02-08  


In [25]:
summarize_columns(df_cleaned, columns=['orders_post', 'analytical_profit_post'], groupby=['entity_id', 'is_customer_holdout'])

,entity_id,is_customer_holdout,column,mean,std,min,max,missing_pct
0,AP_PA,False,orders_post,3.179680,3.687803,1.000000,160.000000,0.0
1,AP_PA,False,analytical_profit_post,4.988012,11.105949,-290.900351,507.243373,0.0
2,AP_PA,True,orders_post,3.086420,3.635444,1.000000,53.000000,0.0
3,AP_PA,True,analytical_profit_post,4.989429,11.162526,-82.548479,227.463396,0.0
4,DJ_CZ,False,orders_post,2.779686,3.303329,1.000000,110.000000,0.0
...,...,...,...,...,...,...,...,...
211,TB_QA,True,analytical_profit_post,6.787450,20.253671,-300.058354,328.749164,0.0
212,YS_TR,False,orders_post,2.798486,3.243357,1.000000,147.000000,0.0
213,YS_TR,False,analytical_profit_post,1.743059,6.652252,-319.292897,483.309145,0.0
214,YS_TR,True,orders_post,2.775602,3.132898,1.000000,58.000000,0.0


In [27]:
from utils.dates import get_iso_week_mondays


get_iso_week_mondays(2025, 
                         date('2025-02-02'), 
                         date('2025-02-03'))

NameError: name 'date' is not defined